In [38]:
import pandas as pd
from datetime import date
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import MinMaxScaler
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout,LSTM,Activation,RepeatVector
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf

In [99]:
df = pd.read_csv('m-hsoldhst6312.txt', sep=' ')

In [100]:
df.head()

,year,mon,hsold,hstart
0,1963,1,42,79.0
1,1963,2,35,89.6
2,1963,3,44,124.8
3,1963,4,52,164.2
4,1963,5,58,172.7


In [101]:
df['Date'] = df['year'].astype(str)+' /'+df['mon'].astype(str)

In [102]:
df_new = df[['Date', 'hsold','hstart']]
df_new

,Date,hsold,hstart
0,1963 /1,42,79.0
1,1963 /2,35,89.6
2,1963 /3,44,124.8
3,1963 /4,52,164.2
4,1963 /5,58,172.7
...,...,...,...
590,2012 /3,34,58.0
591,2012 /4,34,66.8
592,2012 /5,35,67.8
593,2012 /6,34,74.7


In [140]:
data_set = df_new[['hsold','hstart']]
data_set

,hsold,hstart
0,42,79.0
1,35,89.6
2,44,124.8
3,52,164.2
4,58,172.7
...,...,...
590,34,58.0
591,34,66.8
592,35,67.8
593,34,74.7


In [141]:
#Normalizing the data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_dataset = scaler.fit_transform(data_set)
scaled_dataset.shape

(595, 2)

In [142]:
def create_dataset(scaled_data):
    window_size = 6
    x_set =[]
    y_set =[]
    
    for i in range(window_size,len(scaled_data)):
        x_set.append(scaled_data[i-window_size:i,:])
        y_set.append(scaled_data[i,0:1])
        
    return x_set, y_set
    

In [143]:
#get the x data and y data 
x_set, y_set = create_dataset(scaled_dataset)
x_data = np.array(x_set)
y_data = np.array(y_set)
print(x_data.shape)
print(y_data.shape)

(589, 6, 2)
(589, 1)


In [144]:
#Split data
#x_data
train_size = int(len(x_data)*0.8)
test_size_x = len(x_data) - train_size
train_data_x = x_data[:train_size]
test_data_x =  x_data[train_size:]

print(train_data_x.shape)
print(test_data_x.shape)

#y_data
train_size = int(len(y_data)*0.8)
test_size_y = len(y_data) - train_size
train_data_y = y_data[:train_size]
test_data_y =  y_data[train_size:]
print(train_data_y.shape)
print(test_data_y.shape)

(471, 6, 2)
(118, 6, 2)
(471, 1)
(118, 1)


In [145]:
train_data_x.shape[1]
train_data_x.shape[2]

2

In [130]:
model = Sequential()

model.add(LSTM(units = 512, return_sequences = True, input_shape=(train_data_x.shape[1], train_data_x.shape[2])))
model.add(Dropout(0.2))

#model.add(LSTM(units = 256, return_sequences = True))
#model.add(Dropout(0.2))

model.add(LSTM(units = 256, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units = 256))
model.add(Dropout(0.2))

model.add(Dense(units = 1))


model.compile(optimizer = 'adam', loss = 'mean_squared_error')  

In [147]:
#model fit
history = model.fit(train_data_x, train_data_y, epochs = 200, batch_size=80)

Epoch 1/200
6/6 [==============================] - 8s 466ms/step - loss: 0.1637
Epoch 2/200
6/6 [==============================] - 3s 459ms/step - loss: 0.0684
Epoch 3/200
6/6 [==============================] - 4s 627ms/step - loss: 0.0233
Epoch 4/200
6/6 [==============================] - 4s 715ms/step - loss: 0.0223
Epoch 5/200
6/6 [==============================] - 4s 717ms/step - loss: 0.0148
Epoch 6/200
6/6 [==============================] - 4s 606ms/step - loss: 0.0118
Epoch 7/200
6/6 [==============================] - 4s 604ms/step - loss: 0.0098
Epoch 8/200
6/6 [==============================] - 4s 682ms/step - loss: 0.0077
Epoch 9/200
6/6 [==============================] - 4s 702ms/step - loss: 0.0072
Epoch 10/200
6/6 [==============================] - 4s 618ms/step - loss: 0.0069
Epoch 11/200
6/6 [==============================] - 4s 599ms/step - loss: 0.0063
Epoch 12/200
6/6 [==============================] - 4s 686ms/step - loss: 0.0059
Epoch 13/200
6/6 [===================

In [148]:
#Get predictions
y_predicted = model.predict(test_data_x)
#y_predicted

4/4 [==============================] - 2s 131ms/step


In [149]:
test_data_y

array([[0.53271028],
       [0.4953271 ],
       [0.46728972],
       [0.52336449],
       [0.57943925],
       [0.72897196],
       [0.6635514 ],
       [0.75700935],
       [0.81308411],
       [0.73831776],
       [0.79439252],
       [0.65420561],
       [0.63551402],
       [0.52336449],
       [0.51401869],
       [0.64485981],
       [0.76635514],
       [0.96261682],
       [0.8317757 ],
       [0.88785047],
       [0.79439252],
       [0.71028037],
       [0.76635514],
       [0.69158879],
       [0.75700935],
       [0.59813084],
       [0.58878505],
       [0.6728972 ],
       [0.8317757 ],
       [1.        ],
       [0.89719626],
       [0.93457944],
       [0.88785047],
       [0.90654206],
       [0.8411215 ],
       [0.73831776],
       [0.79439252],
       [0.61682243],
       [0.62616822],
       [0.64485981],
       [0.63551402],
       [0.82242991],
       [0.74766355],
       [0.76635514],
       [0.72897196],
       [0.58878505],
       [0.63551402],
       [0.560

In [150]:
data = scaled_dataset[:118,:]
y_pred_act = np.concatenate((y_predicted,data[:118,1:]),axis=1)
y_test_act = np.concatenate((test_data_y,data[:118,1:]),axis=1)


In [151]:
#Calculate RMSE
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(test_data_y, y_predicted)

rmse = np.sqrt(mse)
rmse

0.0856402828940251

In [152]:
Y_tested = scaler.inverse_transform(y_test_act)
Y_predicted = scaler.inverse_transform(y_pred_act)

Y_predicted = Y_predicted[:,0:1]
Y_tested = Y_tested[:,0:1]
Y_tested.shape

(118, 1)

In [153]:
#Calculate RMSE
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(Y_tested,Y_predicted)

rmse = np.sqrt(mse)
rmse

9.163510269660689